In [14]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-5")

In [6]:
llm.invoke("Hi")

AIMessage(content='Hi! How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 82, 'prompt_tokens': 7, 'total_tokens': 89, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CvPjVEAX3IHvKpgGSr697Y2W0HBE5', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b9911-f12f-7c31-a295-a94cd804c65d-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 7, 'output_tokens': 82, 'total_tokens': 89, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 64}})

In [15]:
from langchain.tools import tool
from pydantic import BaseModel

@tool
def write_email(to:str, subject:str, content:str):
    """Write and send an email"""
    return f"Email has been send to {to} with subject : {subject} and content : {content}"

@tool
def schedule_meeting(attendies:list[str], subject:str, duration_minutes:str, preferred_day:str,  start_time:str):
    """Schdule a calender meeting"""
    date_str = preferred_day.strftime("%A, %B %d, %Y")
    return f"Meeting '{subject}' scheduled on {date_str} at {start_time} for {duration_minutes} minutes with {len(attendees)} attendees"

@tool
def check_calendar_availability(day: str) -> str:
    """Check calendar availability for a given day."""
    # Placeholder response - in real app would check actual calendar
    return f"Available times on {day}: 9:00 AM, 2:00 PM, 4:00 PM"

@tool
class Done(BaseModel):
      """E-mail has been sent."""
      done: bool
    

In [16]:
from langgraph.graph import START, END, MessagesState, StateGraph
from typing import Literal
class State(MessagesState):
    email_input:dict
    classification_decision: Literal["ignore", "respond", "notify"]

In [17]:
%load_ext autoreload
%autoreload 2
from email_assistant.prompts import triage_system_prompt, triage_user_prompt, default_triage_instructions, default_background
from rich.markdown import Markdown
from pydantic import BaseModel, Field
from email_assistant.utils import parse_email, format_email_markdown


Markdown(default_triage_instructions)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Emails that are not worth responding to:                                                                           

 • Marketing newsletters and promotional emails                                                                    
 • Spam or suspicious emails                                                                                       
 • CC'd on FYI threads with no direct questions                                                                    

There are also other things that should be known about, but don't require an email response. For these, you should 
notify (using the notify response). Examples of this include:                                                      

 • Team member out sick or on vacation                                                                             
 • Build system notifications or deployments                                                                       
 • Project status updates without action items                                                                     
 • Important company announcements                                                                                 
 • FYI emails that contain relevant information for current projects                                               
 • HR Department deadline reminders                                                                                
 • Subscription status / renewal reminders                                                                         
 • GitHub notifications                                                                                            

Emails that are worth responding to:                                                                               

 • Direct questions from team members requiring expertise                                                          
 • Meeting requests requiring confirmation                                                                         
 • Critical bug reports related to team's projects                                                                 
 • Requests from management requiring acknowledgment                                                               
 • Client inquiries about project status or features                                                               
 • Technical questions about documentation, code, or APIs (especially questions about missing endpoints or         
   features)                                                                                                       
 • Personal reminders related to family (wife / daughter)                                                          
 • Personal reminder related to self-care (doctor appointments, etc)

In [31]:
class RouterSchema(BaseModel):
    """Analyze the unread email and route it according to its content."""

    reasoning: str = Field(
        description="Step-by-step reasoning behind the classification."
    )
    classification: Literal["ignore", "respond", "notify"] = Field(
        description="The classification of an email: 'ignore' for irrelevant emails, "
        "'notify' for important information that doesn't need a response, "
        "'respond' for emails that need a reply",
    )


llm_with_router = llm.with_structured_output(RouterSchema)


In [12]:
from email_assistant.utils import parse_email, format_email_markdown

llm_with_router.invoke("hello")

RouterSchema(reasoning='The message contains only a greeting with no information or request. It likely initiates a conversation and requires a follow-up to clarify their intent.', classification='respond')

In [19]:
email_input = {
    "author": "System Admin <sysadmin@company.com>",
    "to": "Development Team <dev@company.com>",
    "subject": "Scheduled maintenance - database downtime",
    "email_thread": "Hi team,\n\nThis is a reminder that we'll be performing scheduled maintenance on the production database tonight from 2AM to 4AM EST. During this time, all database services will be unavailable.\n\nPlease plan your work accordingly and ensure no critical deployments are scheduled during this window.\n\nThanks,\nSystem Admin Team"
}
    
    

In [20]:
author, to, subject, email_thread = parse_email(email_input)


In [17]:
author

'System Admin <sysadmin@company.com>'

In [21]:
from rich.markdown import Markdown

Markdown(triage_system_prompt)

< Role > Your role is to triage incoming emails based upon instructs and background information below. </ Role >   

< Background > {background}. </ Background >                                                                       

< Instructions > Categorize each email into one of three categories:                                               

 1 IGNORE - Emails that are not worth responding to or tracking                                                    
 2 NOTIFY - Important information that worth notification but doesn't require a response                           
 3 RESPOND - Emails that need a direct response Classify the below email into one of these categories. </          
   Instructions >                                                                                                  

< Rules > {triage_instructions} </ Rules >

In [ ]:
Markdown(default_background)

I'm Lance, a software engineer at LangChain.

In [2]:
Markdown(default_triage_instructions)

Emails that are not worth responding to:                                                                           

 • Marketing newsletters and promotional emails                                                                    
 • Spam or suspicious emails                                                                                       
 • CC'd on FYI threads with no direct questions                                                                    

There are also other things that should be known about, but don't require an email response. For these, you should 
notify (using the notify response). Examples of this include:                                                      

 • Team member out sick or on vacation                                                                             
 • Build system notifications or deployments                                                                       
 • Project status updates without action items                                                                     
 • Important company announcements                                                                                 
 • FYI emails that contain relevant information for current projects                                               
 • HR Department deadline reminders                                                                                
 • Subscription status / renewal reminders                                                                         
 • GitHub notifications                                                                                            

Emails that are worth responding to:                                                                               

 • Direct questions from team members requiring expertise                                                          
 • Meeting requests requiring confirmation                                                                         
 • Critical bug reports related to team's projects                                                                 
 • Requests from management requiring acknowledgment                                                               
 • Client inquiries about project status or features                                                               
 • Technical questions about documentation, code, or APIs (especially questions about missing endpoints or         
   features)                                                                                                       
 • Personal reminders related to family (wife / daughter)                                                          
 • Personal reminder related to self-care (doctor appointments, etc)

In [22]:
system_prompt = triage_system_prompt.format(
        background=default_background,
        triage_instructions=default_triage_instructions
    )

In [23]:
Markdown(triage_user_prompt)

Please determine how to handle the below email thread:                                                             

From: {author} To: {to} Subject: {subject} {email_thread}

In [24]:
user_prompt = triage_user_prompt.format(author=author, to=to, subject=subject, email_thread=email_thread)

In [27]:
Markdown(user_prompt)

Please determine how to handle the below email thread:                                                             

From: System Admin ]8;id=908553;mailto:sysadmin@company.com\sysadmin@company.com]8;;\ To: Development Team ]8;id=859152;mailto:dev@company.com\dev@company.com]8;;\ Subject: Scheduled maintenance -      
database downtime Hi team,                                                                                         

This is a reminder that we'll be performing scheduled maintenance on the production database tonight from 2AM to   
4AM EST. During this time, all database services will be unavailable.                                              

Please plan your work accordingly and ensure no critical deployments are scheduled during this window.             

Thanks, System Admin Team

In [29]:
Markdown(system_prompt)

< Role > Your role is to triage incoming emails based upon instructs and background information below. </ Role >   

< Background >                                                                                                     

I'm Lance, a software engineer at LangChain. . </ Background >                                                     

< Instructions > Categorize each email into one of three categories:                                               

 1 IGNORE - Emails that are not worth responding to or tracking                                                    
 2 NOTIFY - Important information that worth notification but doesn't require a response                           
 3 RESPOND - Emails that need a direct response Classify the below email into one of these categories. </          
   Instructions >                                                                                                  

< Rules >                                                                                                          

Emails that are not worth responding to:                                                                           

 • Marketing newsletters and promotional emails                                                                    
 • Spam or suspicious emails                                                                                       
 • CC'd on FYI threads with no direct questions                                                                    

There are also other things that should be known about, but don't require an email response. For these, you should 
notify (using the notify response). Examples of this include:                                                      

 • Team member out sick or on vacation                                                                             
 • Build system notifications or deployments                                                                       
 • Project status updates without action items                                                                     
 • Important company announcements                                                                                 
 • FYI emails that contain relevant information for current projects                                               
 • HR Department deadline reminders                                                                                
 • Subscription status / renewal reminders                                                                         
 • GitHub notifications                                                                                            

Emails that are worth responding to:                                                                               

 • Direct questions from team members requiring expertise                                                          
 • Meeting requests requiring confirmation                                                                         
 • Critical bug reports related to team's projects                                                                 
 • Requests from management requiring acknowledgment                                                               
 • Client inquiries about project status or features                                                               
 • Technical questions about documentation, code, or APIs (especially questions about missing endpoints or         
   features)                                                                                                       
 • Personal reminders related to family (wife / daughter)                                                          
 • Personal reminder related to self-care (doctor appointments, etc)                                               

</ Rules >

In [33]:
result = llm_with_router.invoke(
        [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
    )

In [36]:
result

RouterSchema(reasoning="This is a system/IT notification about scheduled database maintenance and downtime. It contains important operational info relevant to development work but does not ask for a response or action beyond planning accordingly. Fits the 'build system notifications or deployments' category that should be notified, not responded to.", classification='notify')

In [ ]:
## lets build the agent

